## Tutorial: Off-policy evaluation на синтетических данных
Этот ноутбук демонстрирует, как библиотека `policyscope` сравнивает две рекомендательные политики по логам политики A.

### Методы оценки
- **Replay** — учитываем только случаи, где политика B совпадает с A.
- **IPS** — взвешиваем отклики по отношению вероятностей действия. Требует точных пропенситей.
- **SNIPS** — нормализует IPS и снижает дисперсию.
- **Doubly Robust** — комбинирует модель отклика и IPS; достаточно корректности хотя бы одной из них.

In [1]:
from policyscope.synthetic import SynthConfig, SyntheticRecommenderEnv
from policyscope.policies import make_policy
from policyscope.estimators import train_mu_hat, prepare_piB_taken, replay_value, ips_value, snips_value, dr_value
import pandas as pd

In [2]:
cfg = SynthConfig(n_users=2000, seed=0)
env = SyntheticRecommenderEnv(cfg)
users = env.sample_users()
policyA = make_policy('epsilon_greedy', seed=1, epsilon=0.15)
policyB = make_policy('softmax', seed=2, tau=0.7)
logsA = env.simulate_logs_A(policyA, users)
logsA.head()

,user_id,loyal,age,risk,income,region,age_z,risk_z,income_z,a_A,propensity_A,accept,cltv
0,0,0,39,0.164427,33847.243699,2,-0.083333,-1.342291,-0.151654,0,0.8875,1,534.149918
1,1,1,68,0.794970,28464.133287,1,2.333333,1.179882,-0.496750,1,0.8875,0,605.271806
2,2,1,51,0.173588,106405.425932,1,0.916667,-1.305647,2.130351,0,0.8875,1,1552.634426
3,3,1,21,0.460746,23807.909876,0,-1.583333,-0.157015,-0.852632,2,0.8875,1,751.644903
4,4,0,58,0.129342,22354.664095,2,1.500000,-1.482632,-0.978114,0,0.8875,1,470.883121


In [3]:
mu_accept = train_mu_hat(logsA, target='accept')
piB_taken = prepare_piB_taken(logsA, policyB)
V_replay = replay_value(logsA, policyB.action_argmax(users), target='accept')
V_ips, ess_ips, clip_ips = ips_value(logsA, piB_taken, target='accept', weight_clip=20)
V_snips, ess_snips, clip_snips = snips_value(logsA, piB_taken, target='accept', weight_clip=20)
V_dr, ess_dr, clip_dr = dr_value(logsA, policyB, mu_accept, target='accept', weight_clip=20)
{'replay': V_replay, 'ips': V_ips, 'snips': V_snips, 'dr': V_dr}

{'replay': 0.5228571428571429,
 'ips': 0.44177746015730146,
 'snips': 0.4788211645086519,
 'dr': 0.48146966174506906}

In [4]:
# сравним с истинным значением
V_A_true = env.oracle_value(policyA, users, metric='accept')
V_B_true = env.oracle_value(policyB, users, metric='accept')
{'oracle_A': V_A_true, 'oracle_B': V_B_true, 'oracle_delta': V_B_true - V_A_true}

{'oracle_A': 0.5405, 'oracle_B': 0.5145, 'oracle_delta': -0.026000000000000023}

In [5]:
print('DR оценка:', V_dr)
print('Истинный эффект:', V_B_true - V_A_true)

DR оценка: 0.48146966174506906
Истинный эффект: -0.026000000000000023
